In [ ]:
import glob
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
def crop_resize_image(image):
    std_height, std_width = 68, 120
    image = np.array(image).astype('float32')/255.
    height, width = image.shape[:2]
    if (height/std_height)<=(width/std_width):
        r = height/std_height
    else:
        r = width/std_width
    new_height, new_width = int(std_height*r), int(std_width*r)
    h_center, w_center = height//2, width//2
        
    image = image[int(h_center-(new_height//2)):int(h_center-(new_height//2)+new_height), 
                  int(w_center-(new_width//2)):int(w_center-(new_width//2)+new_width)]
    return image 

In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, 
                                                       samplewise_center=False, 
                                                       featurewise_std_normalization=False, 
                                                       samplewise_std_normalization=False, 
                                                       zca_whitening=False, 
                                                       zca_epsilon=1e-06, 
                                                       rotation_range=0, 
                                                       width_shift_range=0.0, 
                                                       height_shift_range=0.0, 
                                                       brightness_range=None, 
                                                       shear_range=0.0, 
                                                       zoom_range=0.0, 
                                                       channel_shift_range=0.0, 
                                                       fill_mode='nearest', 
                                                       cval=0.0,
                                                       horizontal_flip=False, 
                                                       vertical_flip=False, 
                                                       rescale=None, 
                                                       preprocessing_function=crop_resize_image, 
                                                       data_format=None, 
                                                       validation_split=0.0, 
                                                       dtype=None)

In [ ]:
classes_dict = ['lake', 'mountain', 'ocean', 'river', 'sky']

## Create tf.kears.datagenerator from smaller, target size dicrects.

In [ ]:
datagen_train = datagen.flow_from_directory(directory='/home/jovyan/ta-hsi-datacenter/resize_272_480/', classes = [classes_dict[1]], target_size=(272, 480), batch_size = 16)
datagen_train_2x = datagen.flow_from_directory(directory='/home/jovyan/ta-hsi-datacenter/resize_544_960/', classes = [classes_dict[1]], target_size=(544, 960), batch_size = 4)

In [ ]:
a = next(datagen_train)
plt.imshow(a[0][0])
print(a[1][0])
print(a[0].shape)

In [ ]:
a = next(datagen_train_2x)
plt.imshow(a[0][0])
print(a[1][0])
print(a[0].shape)

In [ ]:
tf.__version__

In [ ]:
import tensorflow.keras.layers as layers

# latent_dim = 100
# height = 256
# width = 480
# channels = 3

def build_generator(latent_dim, output_size):
    filter_num = [128, 128, 256, 256]
    generator_input = keras.Input(shape=(latent_dim,))
# First, transform the input into a 16x16 128-channels feature map
    height, width = output_size
    x = layers.Dense(128 * int(height//16) * int(width//16))(generator_input)
    x = layers.ReLU()(x)
    x = layers.Reshape((int(height//16), int(width//16), 128))(x)

# Then, add a convolution layer
    x = layers.Conv2D(128, 4, padding='same')(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 4, padding='same')(x)
    x = layers.ReLU()(x)

    for i in range(4) :
        x = layers.Conv2DTranspose(filter_num[i], 4, strides=2, padding='same')(x)
        x = layers.ReLU()(x)

    x = layers.Conv2D(3, 4, activation='tanh', padding='same')(x)
    generator = keras.models.Model(generator_input, x)
    generator.summary()
    return generator

In [ ]:
G = build_generator(100, (136, 240))
G.summary()

In [ ]:
def residual_block(res_input):
    x = layers.Conv2D(128, 3, strides=1, padding='same')(res_input)
    x = layers.ReLU()(x)

    x = layers.Conv2D(128, 3, strides=1, padding='same')(x)

    x = layers.add([x, res_input])
    x = layers.ReLU()(x)

    return x

In [ ]:
def generator_up(input_size):
    filter_num = [256, 256]
    height, width, channels = input_size
    up_input = layers.Input(shape = (height, width, channels))
    
    x = layers.Conv2D(64, 5, padding = 'same')(up_input)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(128, 3, padding = 'same')(x)
    x = layers.ReLU()(x)
    
    x = residual_block(x)
    x = residual_block(x)
    
    for i in range(1):
        x = layers.Conv2DTranspose(filter_num[i], 4, strides=2, padding='same')(x)
        x = layers.ReLU()(x)
        
    x = layers.Conv2D(3, 3, activation='tanh', padding='same')(x)
    generator_up = keras.models.Model(up_input, x)
    generator_up.summary()
    
    return generator_up

In [ ]:
G = generator_up((272, 480, 3))
G.summary()

In [ ]:
def build_discriminator(input_size):
    height, width, channels = input_size
    discriminator_input = layers.Input(shape=(height, width, channels))
    x = layers.Conv2D(32, 4, padding = 'same')(discriminator_input)

    x = layers.ReLU()(x)
    x = layers.Conv2D(64, 4, strides = 2, padding = 'same')(x)

    x = layers.ReLU()(x)
    x = layers.Conv2D(64, 4, strides = 2, padding = 'same')(x)

    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 4, strides = 2, padding = 'same')(x)

    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 4, strides = 2, padding = 'same')(x)

    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 4, strides = 2, padding = 'same')(x)

    x = layers.ReLU()(x)
    x = layers.Conv2D(256, 4, strides = 2, padding = 'same')(x)

    x = layers.ReLU()(x)

    x = layers.Flatten()(x)

# One dropout layer - important trick!
    x = layers.Dropout(0.4)(x)

# Classification layer
    x = layers.Dense(1, activation='sigmoid')(x)

    discriminator = keras.models.Model(discriminator_input, x)
    discriminator.summary()
    return discriminator
# To stabilize training, we use learning rate decay
# and gradient clipping (by value) in the optimizer.

In [ ]:
D = build_discriminator((544, 960, 3))
D.summary()

In [ ]:
# Set discriminator weights to non-trainable
# (will only apply to the `gan` model)
def build_GAN(G, D):
    D.trainable = False
    gan_input = G.input
#     gan_input = keras.Input(shape=(latent_dim,))
    gan_output = D(G(gan_input))
    gan = keras.models.Model(gan_input, gan_output)

# gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
#     gan_optimizer = keras.optimizers.Adam(lr=0.0002, beta_1 = 0.5)
#     gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
#     gan.summary()
    return gan

In [ ]:
GAN = build_GAN(G, D)
GAN.summary()

In [ ]:
iterations = 10000
latent_dim = 200
height_1, width_1 = 272, 480
height_2, width_2 = 544, 960

G1 = build_generator(latent_dim, (height_1, width_1))
D1 = build_discriminator((height_1, width_1, 3))
optimizer = keras.optimizers.Adam(lr=0.0001, beta_1 = 0.5)
D1.compile(loss = 'binary_crossentropy', optimizer = optimizer)

G2 = generator_up((height_1, width_1, 3))
D2 = build_discriminator((height_2, width_2, 3))
optimizer = keras.optimizers.Adam(lr=0.0001, beta_1 = 0.5)
D2.compile(loss = 'binary_crossentropy', optimizer = optimizer)

GAN2 = build_GAN(G2, D2)
optimizer = keras.optimizers.Adam(lr=0.0001, beta_1 = 0.5)
GAN2.compile(loss = 'binary_crossentropy', optimizer = optimizer)

In [ ]:
GAN1 = build_GAN(G1, D1)
optimizer = keras.optimizers.Adam(lr=0.0001, beta_1 = 0.5)
GAN1.compile(loss = 'binary_crossentropy', optimizer = optimizer)

## Load weights

In [ ]:
from tensorflow.keras.models import load_model
G1.load_weights('./output_3/Checkpoint/g1.h5')

D1.load_weights('./output_3/Checkpoint/d1.h5')
# optimizer = keras.optimizers.Adam(lr=0.0001, beta_1 = 0.5)
# D1.compile(loss = 'binary_crossentropy', optimizer = optimizer)

In [ ]:
import os
from keras.preprocessing import image
import time

start_time_all = time.time()


save_dir = './output_3/gan_images/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Start training loop
start = 0
start_time = time.time()

low_iteration = 600
high_iteration = 1000
pre_low_step = 55300
pre_high_step = 5700
batch_size = 16
for step in range(iterations):
    for low_step in range(low_iteration):
        real_images = next(datagen_train)[0]
#         stop = start+batch_size
#         real_images = x_train[start: stop]
#         batch_size = real_images.shape[0]
        random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
        
        
        generated_images = G1.predict(random_latent_vectors)
        
        labels = np.concatenate([np.zeros((batch_size, 1)),
                             np.ones((batch_size, 1))])

        labels_real = np.ones((batch_size, 1)) - 0.05*np.abs(np.random.random((batch_size, 1)))
        labels_fake = np.zeros((batch_size, 1)) + 0.05*np.abs(np.random.random((batch_size, 1)))

        d_loss_real = D1.train_on_batch(real_images, labels_real)
        d_loss_fake = D1.train_on_batch(generated_images, labels_fake)
        d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

        random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

        misleading_targets = np.ones((batch_size, 1))

        g_loss = GAN1.train_on_batch(random_latent_vectors, misleading_targets)
#         start += batch_size
#         if start > len(x_train) - batch_size:
#             start = 0

        if low_step % 100 == 0:
            # Save model weights
            GAN1.save_weights('./output_3/gan1.h5')
            G1.save_weights('./output_3/g1.h5')
            D1.save_weights('./output_3/d1.h5')

            # Print metrics
            print('low resolution, discriminator loss at step %s: %s' % (step*low_iteration+low_step+pre_low_step, d_loss))
            print('low resolution, adversarial loss at step %s: %s' % (step*low_iteration+low_step+pre_low_step, g_loss))
            display_grid = np.zeros((4*height_1,width_1,3))
        
            for j in range(4):
                display_grid[j*height_1:(j+1)*height_1,0:width_1,:] = generated_images[j]
        
            img = image.array_to_img((display_grid[:,:,::-1]*127.5)+127.5, scale=False)
            img.save(os.path.join(save_dir, 'low_generated_' + str(step*low_iteration+low_step+pre_low_step) + '.png'))
            print("--- %s seconds ---" % (time.time() - start_time))
            start_time = time.time()
    for high_step in range(high_iteration):
        real_images = next(datagen_train_2x)[0]
        batch_size = real_images.shape[0]
#        real_images = x_train2x[start: stop]
        random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
        
        low_generated_images = G1.predict(random_latent_vectors)
        generated_images = G2.predict(low_generated_images)

        labels_real = np.ones((batch_size, 1)) - 0.05*np.abs(np.random.random((batch_size, 1)))
        labels_fake = np.zeros((batch_size, 1)) + 0.05*np.abs(np.random.random((batch_size, 1)))

        d_loss_real = D2.train_on_batch(real_images, labels_real)
        d_loss_fake = D2.train_on_batch(generated_images, labels_fake)
        d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

        random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
        G1_out = G1.predict(random_latent_vectors)
        misleading_targets = np.ones((batch_size, 1))
        
        g_loss = GAN2.train_on_batch(G1_out, misleading_targets)
    
        if high_step % 100 == 0:
            # Save model weights
            GAN2.save_weights('./output_3/gan2.h5')
            G2.save_weights('./output_3/g2.h5')
            D2.save_weights('./output_3/d2.h5')
            # Print metrics
            print('high resolution, discriminator loss at step %s: %s' % (step*high_iteration+high_step+pre_high_step, d_loss))
            print('high resolution, adversarial loss at step %s: %s' % (step*high_iteration+high_step+pre_high_step, g_loss))
            display_grid = np.zeros((4*height_2,width_2,3))
        
            for j in range(4):
                display_grid[j*height_2:(j+1)*height_2,0:width_2,:] = generated_images[j]
        
            img = image.array_to_img((display_grid[:,:,::-1]*127.5)+127.5, scale=False)
            img.save(os.path.join(save_dir, 'high_generated_' + str(step*high_iteration+high_step+pre_high_step) + '.png'))
            print("--- %s seconds ---" % (time.time() - start_time))
            start_time = time.time()
        
print("--- %s seconds ---" % (time.time() - start_time_all))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(display_grid[:,:,::-1])